In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import csv

In [2]:
df = pd.read_csv('cleandataset.csv', header=0)
#fill in missing papers
df['paper'].fillna(method='ffill', inplace=True)
df.head()

,Unnamed: 0,aid,paper,author,date,url,content,clean_content
0,0,wsj0,wsj,"['Natalie Andrews', 'Louise Radnofsky', 'Krist...",NaN,https://www.wsj.com/articles/trump-calls-meeti...,"President Donald Trump, in a meeting with lawm...","['president', 'donald', 'trump', 'meeting', 'l..."
1,1,wsj1,wsj,"['Sarah Krouse', 'Sarah.Krouse Wsj.Com']",NaN,https://www.wsj.com/articles/blackrock-goes-pu...,BlackRock is the largest shareholder in gun ma...,"['blackrock', 'largest', 'shareholder', 'gun',..."
2,2,wsj2,wsj,"['Sarah Nassauer', 'Sarah.Nassauer Wsj.Com']",NaN,https://www.wsj.com/articles/gun-makers-strate...,"Vista Outdoor Inc., an ammunition and gun manu...","['vista', 'outdoor', 'inc.', 'ammunition', 'gu..."
3,3,wsj3,wsj,[],NaN,http://www.wsj.com/video/how-student-survivors...,2:57\n\nMyanmar's Rohingya Refugees Face Prote...,"['2:57', 'myanmar', 'rohingya', 'refugee', 'fa..."
4,4,wsj4,wsj,"['Heather Haddon', 'Google', 'Heather.Haddon W...",NaN,https://www.wsj.com/articles/kroger-to-stop-se...,In the wake of the school shooting in Parkland...,"['wake', 'school', 'shooting', 'parkland', 'fl..."


In [3]:
#sentiment analyzer
def sentanal(tokens):
    sents=[]
    for tok in tokens:
        blob=TextBlob(tok, analyzer=NaiveBayesAnalyzer())
        sents.append(blob.sentiment)
    return sents

In [4]:
#analyze just the first line of each article
#this could and probably should be much faster!!!
s1=[]
for cont in df['content']:
    myl=cont.split('.')
    line1=myl[0]
    s1.append(line1)

In [7]:
mysent=sentanal(s1)

In [8]:
#inspect
mysent[:5]

[Sentiment(classification='pos', p_pos=0.6420442386214867, p_neg=0.3579557613785149),
 Sentiment(classification='pos', p_pos=0.5088077765477111, p_neg=0.4911922234522903),
 Sentiment(classification='pos', p_pos=0.6499999999999997, p_neg=0.3500000000000005),
 Sentiment(classification='pos', p_pos=0.9763323364435164, p_neg=0.023667663556481474),
 Sentiment(classification='pos', p_pos=0.739435833091521, p_neg=0.26056416690847856)]

In [9]:
#make a dataframe out of the results
#is there a better way to do this - more directly? Omit the listing step?
sentlist=[]
poslist=[]
neglist=[]
for item in mysent:
    sentlist.append(item.classification)
    poslist.append(item.p_pos)
    neglist.append(item.p_neg)
df['sent']=sentlist
df['p_pos']=poslist
df['p_neg']=neglist

In [10]:
df.head(5)

,Unnamed: 0,aid,paper,author,date,url,content,clean_content,sent,p_pos,p_neg
0,0,wsj0,wsj,"['Natalie Andrews', 'Louise Radnofsky', 'Krist...",NaN,https://www.wsj.com/articles/trump-calls-meeti...,"President Donald Trump, in a meeting with lawm...","['president', 'donald', 'trump', 'meeting', 'l...",pos,0.642044,0.357956
1,1,wsj1,wsj,"['Sarah Krouse', 'Sarah.Krouse Wsj.Com']",NaN,https://www.wsj.com/articles/blackrock-goes-pu...,BlackRock is the largest shareholder in gun ma...,"['blackrock', 'largest', 'shareholder', 'gun',...",pos,0.508808,0.491192
2,2,wsj2,wsj,"['Sarah Nassauer', 'Sarah.Nassauer Wsj.Com']",NaN,https://www.wsj.com/articles/gun-makers-strate...,"Vista Outdoor Inc., an ammunition and gun manu...","['vista', 'outdoor', 'inc.', 'ammunition', 'gu...",pos,0.650000,0.350000
3,3,wsj3,wsj,[],NaN,http://www.wsj.com/video/how-student-survivors...,2:57\n\nMyanmar's Rohingya Refugees Face Prote...,"['2:57', 'myanmar', 'rohingya', 'refugee', 'fa...",pos,0.976332,0.023668
4,4,wsj4,wsj,"['Heather Haddon', 'Google', 'Heather.Haddon W...",NaN,https://www.wsj.com/articles/kroger-to-stop-se...,In the wake of the school shooting in Parkland...,"['wake', 'school', 'shooting', 'parkland', 'fl...",pos,0.739436,0.260564


In [11]:
#this is too big - drop the content
df=df.drop('content', axis=1)
df=df.drop('clean_content', axis=1)

In [12]:
df.head(5)

,Unnamed: 0,aid,paper,author,date,url,sent,p_pos,p_neg
0,0,wsj0,wsj,"['Natalie Andrews', 'Louise Radnofsky', 'Krist...",NaN,https://www.wsj.com/articles/trump-calls-meeti...,pos,0.642044,0.357956
1,1,wsj1,wsj,"['Sarah Krouse', 'Sarah.Krouse Wsj.Com']",NaN,https://www.wsj.com/articles/blackrock-goes-pu...,pos,0.508808,0.491192
2,2,wsj2,wsj,"['Sarah Nassauer', 'Sarah.Nassauer Wsj.Com']",NaN,https://www.wsj.com/articles/gun-makers-strate...,pos,0.650000,0.350000
3,3,wsj3,wsj,[],NaN,http://www.wsj.com/video/how-student-survivors...,pos,0.976332,0.023668
4,4,wsj4,wsj,"['Heather Haddon', 'Google', 'Heather.Haddon W...",NaN,https://www.wsj.com/articles/kroger-to-stop-se...,pos,0.739436,0.260564


In [14]:
#save the sentiments out
outfile = open('sentiments.csv', 'w')
df.to_csv(outfile)